Part-1 -- Naive Bayes Implementation

In [139]:
import pandas as pd
df = pd.read_csv('weather/weather.csv')
df.head()

,Outlook,Temp,Humidity,Windy,Play
0,rainy,hot,high,0,0
1,rainy,hot,high,1,0
2,overcast,hot,high,0,1
3,sunny,mild,high,0,1
4,sunny,cool,normal,0,1


In [140]:
X_train = df.iloc[:,0:4]
X_train

,Outlook,Temp,Humidity,Windy
0,rainy,hot,high,0
1,rainy,hot,high,1
2,overcast,hot,high,0
3,sunny,mild,high,0
4,sunny,cool,normal,0
5,sunny,cool,normal,1
6,overcast,cool,normal,1
7,rainy,mild,high,0
8,rainy,cool,normal,0
9,sunny,mild,normal,0


In [141]:
y_train = df.iloc[:,-1]
y_train

,Play
0,0
1,0
2,1
3,1
4,1
5,0
6,1
7,0
8,1
9,1


In [142]:
df_test = pd.read_csv('weather/weather_test.csv')
df_test

,Outlook,Temp,Humidity,Windy,Play
0,rainy,cool,high,1,0
1,overcast,mild,normal,0,1


In [143]:
X_test = df_test.iloc[:,0:4]
X_test

,Outlook,Temp,Humidity,Windy
0,rainy,cool,high,1
1,overcast,mild,normal,0


In [144]:
y_test = df_test.iloc[:,-1]
y_test

,Play
0,0
1,1


In [145]:
#Prior Probability
import numpy as np
train_size = X_train.shape[0]
class_priors={}
for outcome in np.unique(y_train):
    outcome_count = sum(y_train == outcome)
    class_priors[outcome] = outcome_count / train_size
print(class_priors)

{np.int64(0): 0.35714285714285715, np.int64(1): 0.6428571428571429}


In [146]:
#Computing Likelihood
features=list(X_train.columns)
Likelihoods={}
for outcome in np.unique(y_train):
    outcome_count = sum(y_train == outcome)
    for feature in features:
         for feat_value in np.unique(X_train[feature]):
              count=0
              for i in range(len(X_train)):
                if(X_train[feature][i]==feat_value and y_train[i]==outcome):
                    count=count+1
              Likelihoods[(feature,feat_value,outcome)] = (count+1) / (outcome_count+(len(features)))

In [147]:
Likelihoods

{('Outlook', 'overcast', np.int64(0)): 0.1111111111111111,
 ('Outlook', 'rainy', np.int64(0)): 0.4444444444444444,
 ('Outlook', 'sunny', np.int64(0)): 0.3333333333333333,
 ('Temp', 'cool', np.int64(0)): 0.2222222222222222,
 ('Temp', 'hot', np.int64(0)): 0.3333333333333333,
 ('Temp', 'mild', np.int64(0)): 0.3333333333333333,
 ('Humidity', 'high', np.int64(0)): 0.5555555555555556,
 ('Humidity', 'normal', np.int64(0)): 0.2222222222222222,
 ('Windy', np.int64(0), np.int64(0)): 0.3333333333333333,
 ('Windy', np.int64(1), np.int64(0)): 0.4444444444444444,
 ('Outlook', 'overcast', np.int64(1)): 0.38461538461538464,
 ('Outlook', 'rainy', np.int64(1)): 0.23076923076923078,
 ('Outlook', 'sunny', np.int64(1)): 0.3076923076923077,
 ('Temp', 'cool', np.int64(1)): 0.3076923076923077,
 ('Temp', 'hot', np.int64(1)): 0.23076923076923078,
 ('Temp', 'mild', np.int64(1)): 0.38461538461538464,
 ('Humidity', 'high', np.int64(1)): 0.3076923076923077,
 ('Humidity', 'normal', np.int64(1)): 0.5384615384615384,


In [148]:
a=len(np.unique(y_train))
a

2

In [149]:
prob=np.ones((a,len(X_test)),dtype=float)
prob

array([[1., 1.],
       [1., 1.]])

In [150]:
for outcome in (np.unique(y_train)):
    outcome_count = sum(y_train==outcome)
    for feature in features:
        for i in range(len(X_test)):
            if(feature,X_test[feature][i],outcome) in Likelihoods.keys():
                prob[outcome][i]=prob[outcome][i]*Likelihoods[(feature,X_test[feature][i],outcome)]
            else:
                prob[outcome][i]=prob[outcome][i]*(1/(outcome_count+len(features)))

In [151]:
#Multiplying probabilities with class prior probabilities
for i in range(prob.shape[0]):
    prob[i][:]=prob[i][:]*class_priors[i]

In [152]:
#Predict the labels
y_label = np.zeros(len(y_test))
for i in range(len(X_test)):
    if(prob[1,i]>=prob[0,i]):
        y_label[i]=1

In [153]:
#Performance Evaluation
from sklearn import metrics
print(metrics.classification_report(y_test,y_label))
print(metrics.confusion_matrix(y_test,y_label))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

[[1 0]
 [0 1]]


Part-2  -- Naive Bayes Classifier -- Using Titanic Dataset

In [154]:
import pandas as pd
df = pd.read_csv('titanic/titanic_train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [155]:
#df.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'], axis='columns', inplace=True)
# df.drop(columns=['PassengerId',	'Name',	'SibSp',	'Parch',	'Ticket', 'Cabin', 'Embarked'],inplace=True)
df.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [156]:
target = df.Survived
inputs = df.drop('Survived',axis='columns')

In [157]:
inputs.head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [158]:
target.head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [159]:
#Convert from bool to int
# dummies = pd.get_dummies(inputs.Sex).astype(int)
# dummies = dummies.replace({True: 1, False: 0})
# dummies = dummies.applymap(int)
# dummies = dummies.infer_objects().astype(int)
dummies = pd.get_dummies(inputs.Sex,dtype=int)
dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [160]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head()

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0
3,1,female,35.0,53.1000,1,0
4,3,male,35.0,8.0500,0,1


In [161]:
inputs.drop('Sex',axis=1,inplace=True)
inputs.head()

,Pclass,Age,Fare,female,male
0,3,22.0,7.2500,0,1
1,1,38.0,71.2833,1,0
2,3,26.0,7.9250,1,0
3,1,35.0,53.1000,1,0
4,3,35.0,8.0500,0,1


In [162]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [163]:
inputs.Age[:10]

,Age
0,22.0
1,38.0
2,26.0
3,35.0
4,35.0
5,NaN
6,54.0
7,2.0
8,27.0
9,14.0


In [164]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head(10)

,Pclass,Age,Fare,female,male
0,3,22.000000,7.2500,0,1
1,1,38.000000,71.2833,1,0
2,3,26.000000,7.9250,1,0
3,1,35.000000,53.1000,1,0
4,3,35.000000,8.0500,0,1
5,3,29.699118,8.4583,0,1
6,1,54.000000,51.8625,0,1
7,3,2.000000,21.0750,0,1
8,3,27.000000,11.1333,1,0
9,2,14.000000,30.0708,1,0


In [165]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)

In [166]:
len(X_train)

712

In [167]:
len(X_test)

179

In [168]:
len(inputs)

891

In [169]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [170]:
model.fit(X_train,y_train)

GaussianNB()

In [171]:
model.score(X_test,y_test)

0.7653631284916201

In [172]:
X_test[:10]

,Pclass,Age,Fare,female,male
659,1,58.000000,113.2750,0,1
576,2,34.000000,13.0000,1,0
261,3,3.000000,31.3875,0,1
346,2,40.000000,13.0000,1,0
32,3,29.699118,7.7500,1,0
684,2,60.000000,39.0000,0,1
766,1,29.699118,39.6000,0,1
119,3,2.000000,31.2750,1,0
416,2,34.000000,32.5000,1,0
497,3,29.699118,15.1000,0,1


In [173]:
y_test[:10]

,Survived
659,0
576,1
261,1
346,1
32,1
684,0
766,0
119,0
416,1
497,0


In [174]:
model.predict(X_test[:10])

array([1, 1, 0, 1, 1, 0, 0, 1, 1, 0])

In [175]:
model.predict_proba(X_test[:10])

array([[0.15989249, 0.84010751],
       [0.0386489 , 0.9613511 ],
       [0.97846945, 0.02153055],
       [0.03925659, 0.96074341],
       [0.07870364, 0.92129636],
       [0.96889632, 0.03110368],
       [0.91235586, 0.08764414],
       [0.03224802, 0.96775198],
       [0.03403899, 0.96596101],
       [0.99126526, 0.00873474]])

Part-3
    -- Naive Bayes Classifier -- Using Spam Classification

In [176]:
import pandas as pd
df = pd.read_csv('Spam/spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [177]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [178]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [179]:
X=df.Message

In [180]:
y=df.spam

In [181]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [182]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [183]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB()

In [184]:
emails = [
    'Hey moin, can we get together to watch football game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

In [185]:
X_test_count = v.transform(X_test)
model.score(X_test_count,y_test)

0.9834888729361091

In [186]:
#Using Pipeline
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

In [187]:
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [188]:
clf.score(X_test,y_test)

0.9834888729361091

In [189]:
clf.predict(emails)

array([0, 1])